## Import Libraries

In [1]:
#!/usr/bin/env python
# coding: utf-8
import os
import numpy as np
import pandas as pd
import requests
from datetime import datetime
import json

In [2]:
# Variables
season = "2021-22"

## Import files

In [3]:
all_events = pd.read_csv(r'./data/' + season + '/Cleaned_data/all_events.csv', parse_dates= ["deadline_time"])
all_players = pd.read_csv(r'./data/' + season + '/Merged_data/all_players_merged.csv', parse_dates= ["news_added"]) 



## Functions

In [4]:
#### Connect to FPL API
def get(url):
    response = requests.get(url)
    return json.loads(response.content)

## My Team

In [5]:
#### Get my fantasy team lineup
now = datetime.now()
my_team_id = '1341730'
current_gw = all_events[all_events['deadline_time'] < now]['gameweek_id'].max()

my_team_url = 'https://fantasy.premierleague.com/api/entry/{}/event/{}/picks/'.format(my_team_id,str(current_gw))

my_team = get(my_team_url)['picks']
my_team = pd.DataFrame(my_team)


In [6]:
all_players.columns

Index(['chance_of_playing_next_round', 'chance_of_playing_this_round',
       'player_code', 'cost_change_event', 'cost_change_event_fall',
       'cost_change_start', 'cost_change_start_fall', 'dreamteam_count',
       'player_type_id', 'ep_next', 'ep_this', 'event_points', 'first_name',
       'form', 'player_id', 'in_dreamteam', 'news', 'news_added', 'now_cost',
       'photo', 'PPG', 'second_name', 'selected_by_percent', 'status',
       'team_id', 'team_code', 'total_points', 'transfers_in',
       'transfers_in_event', 'transfers_out', 'transfers_out_event',
       'value_form', 'value_season', 'web_name', 'total_minutes',
       'goals_scored', 'assists', 'clean_sheets', 'goals_conceded',
       'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards',
       'red_cards', 'saves', 'bonus', 'bps', 'influence', 'creativity',
       'threat', 'ict_index', 'influence_rank', 'influence_rank_type',
       'creativity_rank', 'creativity_rank_type', 'threat_rank',
       'thre

In [7]:
#all_players = all_players[['chance_of_playing_next_round', 'chance_of_playing_this_round', 'position_id', 'ep_next','ep_this', 'player_id', 'player_news', 'news_added', 'player_price', 'selected_by_percent', 'player_status','team_id', 'web_name']]

all_players.head()

,chance_of_playing_next_round,chance_of_playing_this_round,player_code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,player_type_id,ep_next,...,penalties_order,penalties_text,PP90,PPMM,VAPM,team_strength,team_short_name,strength_overall_home,strength_overall_away,position_name_short
0,100.0,100.0,80201,0,0,-5,5,0,1,0.0,...,NaN,NaN,1.333333,0.288889,-0.155556,4,ARS,1210,1270,GKP
1,0.0,0.0,115918,0,0,0,0,0,1,0.0,...,NaN,NaN,NaN,0.000000,-0.500000,4,ARS,1210,1270,GKP
2,0.0,0.0,463748,0,0,0,0,0,1,0.0,...,NaN,NaN,NaN,0.000000,-0.500000,4,ARS,1210,1270,GKP
3,NaN,NaN,225321,0,0,6,-6,3,1,0.0,...,NaN,NaN,5.026596,0.980392,0.588235,4,ARS,1210,1270,GKP
4,NaN,NaN,220682,0,0,-1,1,0,1,0.0,...,NaN,NaN,NaN,0.000000,-0.512821,4,ARS,1210,1270,GKP


In [8]:
my_team

,element,position,multiplier,is_captain,is_vice_captain
0,559,1,1,False,False
1,256,2,1,False,False
2,370,3,1,False,False
3,237,4,1,False,False
4,584,5,1,False,False
5,681,6,1,False,False
6,22,7,1,False,False
7,233,8,3,True,False
8,420,9,1,False,True
9,450,10,1,False,False


In [9]:
my_team = my_team.merge(all_players, how = 'left', left_on="element", right_on="player_id")
my_team.drop(["element","position"], axis = 1, inplace = True)

In [10]:
my_team.sort_values(by=['player_type_id' , 'selected_by_percent'])['web_name']



11                 Cox
0             Ramsdale
4            Cucurella
14              Alonso
2             Reguilón
1              Cancelo
3     Alexander-Arnold
5             Coutinho
6                 Saka
12                Jota
8                Bowen
7                Salah
10                King
13             Watkins
9               Dennis
Name: web_name, dtype: object

In [11]:
my_team.to_csv (r'./data/' + season + '/Team_data/my_team.csv', index = False, header=True)

In [12]:
my_team

,multiplier,is_captain,is_vice_captain,chance_of_playing_next_round,chance_of_playing_this_round,player_code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,...,penalties_order,penalties_text,PP90,PPMM,VAPM,team_strength,team_short_name,strength_overall_home,strength_overall_away,position_name_short
0,1,False,False,NaN,NaN,225321,0,0,6,-6,...,NaN,NaN,5.026596,0.980392,0.588235,4,ARS,1210,1270,GKP
1,1,False,False,100.0,100.0,121145,-1,1,11,-11,...,NaN,NaN,5.952492,0.830986,0.549296,5,MCI,1350,1370,DEF
2,1,False,False,100.0,100.0,199249,0,0,3,-3,...,NaN,NaN,4.864865,0.773585,0.396226,4,TOT,1210,1270,DEF
3,1,False,False,100.0,100.0,169187,0,0,10,-10,...,NaN,NaN,7.136564,0.823529,0.588235,5,LIV,1330,1350,DEF
4,1,False,False,NaN,NaN,179268,0,0,1,-1,...,NaN,NaN,3.751954,0.705882,0.313725,3,BHA,1130,1130,DEF
5,1,False,False,NaN,NaN,84583,0,0,3,-3,...,NaN,NaN,7.414773,0.794521,0.520548,3,AVL,1130,1160,MID
6,1,False,False,100.0,100.0,223340,0,0,0,0,...,NaN,NaN,5.469613,0.707692,0.400000,4,ARS,1210,1270,MID
7,3,True,False,100.0,100.0,118748,0,0,6,-6,...,2.0,NaN,9.030245,0.664122,0.511450,5,LIV,1330,1350,MID
8,1,False,True,NaN,NaN,178186,0,0,6,-6,...,NaN,NaN,6.045987,0.802817,0.521127,4,WHU,1200,1200,MID
9,1,False,False,100.0,100.0,230251,0,0,11,-11,...,NaN,NaN,5.297030,0.770492,0.442623,2,WAT,1050,1050,FWD
